# EV Market Analysis in India

## 1. Data Loading and Initial Exploration

This notebook will be used to analyze the Electric Vehicle market in India. We will start by loading the `EVMarket-India_data.csv` file and performing an initial exploration of the data.


In [ ]:
import pandas as pd

df = pd.read_csv('EVMarket-India_data.csv')
print(df.head())
print(df.info())


## 2. Exploratory Data Analysis (EDA)

We will now perform some exploratory data analysis to understand the dataset better. This includes checking for missing values, descriptive statistics, and distributions of key features.


In [ ]:
print(df.isnull().sum())
print(df.describe())


## 3. Feature Analysis and Correlation

We will now analyze the distributions of key numerical features and examine correlations between them. This will help us identify important factors for segmentation.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Distribution of numerical features
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 12))
sns.histplot(df["AccelSec"], kde=True, ax=axes[0, 0])
sns.histplot(df["TopSpeed_KmH"], kde=True, ax=axes[0, 1])
sns.histplot(df["Range_Km"], kde=True, ax=axes[0, 2])
sns.histplot(df["Efficiency_WhKm"], kde=True, ax=axes[1, 0])
sns.histplot(df["FastCharge_KmH"], kde=True, ax=axes[1, 1])
sns.histplot(df["PriceEuro"], kde=True, ax=axes[1, 2])
plt.tight_layout()
plt.savefig("numerical_feature_distributions.png")
plt.clf()

# Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.select_dtypes(include=["int64", "float64"]).corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Matrix of Numerical Features")
plt.savefig("correlation_matrix.png")
plt.clf()


## 4. Market Segmentation using Clustering

We will use K-Means clustering to identify potential market segments based on vehicle characteristics and price. Before clustering, we need to scale the numerical features.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# Select numerical features for clustering
features = ["AccelSec", "TopSpeed_KmH", "Range_Km", "Efficiency_WhKm", "PriceEuro"]
X = df[features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determine the optimal number of clusters using the elbow method
Sum_of_squared_distances = []
K = range(1, 10)
for k in K:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km = km.fit(X_scaled)
    Sum_of_squared_distances.append(km.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(K, Sum_of_squared_distances, "bx-")
plt.xlabel("k")
plt.ylabel("Sum of squared distances")
plt.title("Elbow Method For Optimal k")
plt.savefig("/home/ubuntu/elbow_method.png")
plt.clf()


Based on the elbow method, we will choose 3 as the optimal number of clusters.


In [ ]:
# Apply K-Means clustering with optimal k
k = 3
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Analyze cluster characteristics
cluster_centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=features)
print(cluster_centers)

# Visualize clusters
sns.pairplot(df, hue="Cluster", vars=features)
plt.suptitle("Pair Plot of Features by Cluster", y=1.02)
plt.savefig("/home/ubuntu/cluster_pairplot.png")
plt.clf()


## 5. Cluster Analysis and Segment Profiling

Based on the cluster centers, we can analyze the characteristics of each segment and give them a descriptive name.


In [ ]:
# Analyze cluster characteristics (referencing the output from the previous step)
# Cluster 0: Low Price, Moderate Range, Moderate Performance
# Cluster 1: High Price, High Performance, High Range, High Efficiency, Fast Charging
# Cluster 2: Moderate Price, Moderate Performance, Moderate Range, Moderate Efficiency, Mixed Charging

# Add cluster labels to the original dataframe for further analysis
df["Cluster"] = kmeans.labels_

# Analyze other features within each cluster (e.g., BodyStyle, Segment, PowerTrain, PlugType)
for cluster_id in range(k):
    print(f"\nCluster {cluster_id} Analysis:")
    print(df[df["Cluster"] == cluster_id][[["Brand", "Model", "BodyStyle", "Segment", "PowerTrain", "PlugType"]]].value_counts())


## 6. Strategic Analysis and Market Entry Strategy

Based on the identified clusters and the Innovation Adoption Life Cycle, we can formulate a market entry strategy.

### 6.1. Segment Profiling and Target Audience

From the cluster analysis, we can define the characteristics of each segment:

*   **Cluster 0: Budget-Conscious Commuters**
    *   **Vehicle Characteristics:** Lower price, moderate range, moderate performance, often smaller body styles (e.g., Hatchback, Sedan).
    *   **Inferred Demographics:** Middle-income individuals, urban and semi-urban dwellers, possibly first-time EV buyers, or those looking for a second car for city commutes.
    *   **Inferred Psychographics:** Practical, value-conscious, environmentally aware but budget-limited, early majority/late majority adopters.
    *   **Inferred Behavioral:** Primarily use vehicles for daily commuting, short-distance travel, prioritize running costs and affordability.

*   **Cluster 1: Performance & Range Seekers**
    *   **Vehicle Characteristics:** Higher price, high performance (quick acceleration, high top speed), long range, high efficiency, often SUVs or larger sedans.
    *   **Inferred Demographics:** High-income individuals, urban dwellers, tech-savvy, early adopters/early majority.
    *   **Inferred Psychographics:** Status-conscious, prioritize performance and advanced technology, willing to pay a premium for features and range, environmentally conscious but not at the expense of performance.
    *   **Inferred Behavioral:** Frequent travelers, desire for a primary vehicle, early adopters of new technology, may have access to private charging.

*   **Cluster 2: Luxury & Premium Segment**
    *   **Vehicle Characteristics:** Very high price, top-tier performance, very long range, luxury features, often premium sedans or large SUVs.
    *   **Inferred Demographics:** Affluent individuals, residing in metropolitan areas, established early adopters/innovators.
    *   **Inferred Psychographics:** Exclusivity-seeking, prioritize luxury, brand image, and cutting-edge technology, not price-sensitive, innovators.
    *   **Inferred Behavioral:** Own multiple vehicles, early adopters of new technology, likely have home charging solutions, use vehicles for long-distance travel and leisure.

### 6.2. Location Analysis for Early Market Entry (Innovation Adoption Life Cycle)

Considering the Innovation Adoption Life Cycle, the initial target market should focus on **Innovators** and **Early Adopters**. These groups are more willing to take risks, are less price-sensitive, and are keen on new technologies. Based on the segment profiling, **Cluster 2 (Luxury & Premium Segment)** and **Cluster 1 (Performance & Range Seekers)** align best with these adopter categories.

Therefore, the most suitable locations for early market entry would be major metropolitan cities in India with a high concentration of affluent and tech-savvy individuals, and relatively better EV infrastructure. These include:

*   **Tier-1 Cities:** Mumbai, Delhi-NCR, Bangalore, Chennai, Hyderabad, Pune.
    *   **Justification:** These cities have a higher disposable income, a larger population of early adopters, better existing charging infrastructure (or potential for rapid development), and a greater awareness of global trends and environmental concerns.

### 6.3. Strategic Pricing Range

The strategic pricing range should align with the target segments (Innovators and Early Adopters) and their psychographics. Given that these segments are less price-sensitive and value performance, luxury, and technology, the pricing should reflect a premium offering.

*   **For Cluster 2 (Luxury & Premium Segment):** Pricing should be at the higher end, reflecting the exclusivity, advanced features, and superior performance. This could range from **€70,000 to €200,000+** (or equivalent in INR), targeting the top 1-2% of the market.

*   **For Cluster 1 (Performance & Range Seekers):** Pricing should be competitive within the premium segment, offering a balance of performance, range, and features. This could range from **€40,000 to €70,000** (or equivalent in INR), targeting the next 5-10% of the market.

It's crucial to note that while the initial focus is on these segments, the long-term strategy should involve gradually introducing more affordable models (targeting Cluster 0 - Budget-Conscious Commuters) to cross the chasm and reach the early and late majority as the market matures and infrastructure develops.

### 6.4. Unavailability of Proper Datasets and Decision Making

In the event of unavailability of proper datasets for psychographic and behavioral factors, decisions can be based on:

*   **Qualitative Research:** Conducting in-depth interviews, focus groups, and surveys with potential customers in target cities to understand their motivations, preferences, and pain points regarding EV adoption. This can provide rich insights into psychographics and behaviors.
*   **Expert Interviews:** Consulting with industry experts, automotive analysts, and thought leaders in the Indian EV space to gather their perspectives and insights on market trends and consumer behavior.
*   **Proxy Data and Analogies:** Utilizing data from similar markets (e.g., other developing economies with nascent EV markets) as a proxy, while carefully considering cultural and economic differences. Drawing analogies from the adoption of other new technologies in India (e.g., smartphones, internet) can also provide valuable insights.
*   **Pilot Programs and A/B Testing:** Launching small-scale pilot programs in selected areas to test different product offerings, pricing strategies, and marketing messages. A/B testing various approaches can provide data-driven insights even with limited initial datasets.
*   **Observational Studies:** Observing consumer behavior in existing EV ecosystems (e.g., charging stations, EV showrooms) to understand real-world usage patterns and preferences.
*   **Academic Research and White Papers:** Leveraging existing academic research, white papers, and reports on consumer behavior, technology adoption, and the automotive industry in India.

To ensure decisions are as accurate and unbiased as possible:

*   **Triangulation:** Combining insights from multiple data sources (qualitative, quantitative, expert opinions) to cross-validate findings and reduce bias.
*   **Hypothesis Testing:** Formulating clear hypotheses about target segments and testing them rigorously with available data and research methods.
*   **Iterative Approach:** Adopting an iterative approach to strategy development, where initial assumptions are continuously refined and validated with new data and market feedback.
*   **Transparency and Documentation:** Clearly documenting all assumptions, data sources, and decision-making processes to ensure transparency and allow for future review and adjustment.
*   **Diversity in Research Team:** Ensuring a diverse research team with varied perspectives to minimize inherent biases in data interpretation and strategy formulation.
